In [ ]:
# === Imports ===
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from collections import defaultdict
import numpy as np
import os

# === Mount Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

# === Set paths ===
base_path = '/content/drive/MyDrive/wheat_data'
train_path = os.path.join(base_path, 'train')
val_path = os.path.join(base_path, 'valid')
test_path = os.path.join(base_path, 'test')

# === Image settings ===
IMG_SIZE = (224, 224)  # MobileNetV2 default
BATCH_SIZE = 32
EPOCHS = 5  # You can increase to 10–15 if accuracy improves

# === Data generators ===
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    train_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True
)

val_gen = datagen.flow_from_directory(
    val_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)

test_gen = datagen.flow_from_directory(
    test_path, target_size=IMG_SIZE, batch_size=1, class_mode='categorical', shuffle=False
)

# === Build MobileNetV2 model ===
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# === Freeze base model layers (transfer learning) ===
for layer in base_model.layers:
    layer.trainable = False

# === Compile and train ===
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, verbose=1)

# === Predict on test set ===
test_gen.reset()
y_pred_probs = model.predict(test_gen, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_gen.classes

# === Clean folder names like 'aphid_test' to 'Aphid' ===
def clean_class_name(name):
    return name.replace('_test', '').replace('_valid', '').replace('_train', '').replace('_', ' ').title()

true_class_names = [clean_class_name(name) for name in test_gen.class_indices.keys()]

# === Accuracy report ===
correct_per_class = defaultdict(int)
total_per_class = defaultdict(int)

for i in range(len(y_true)):
    true_class = y_true[i]
    pred_class = y_pred[i]
    class_name = true_class_names[true_class]
    total_per_class[class_name] += 1
    if pred_class == true_class:
        correct_per_class[class_name] += 1

# === Print results table ===
print(f"\n{'Class':<30}{'Accuracy (%)':<15}{'Correct':<10}{'Total'}")
for class_name in sorted(true_class_names):
    correct = correct_per_class[class_name]
    total = total_per_class[class_name]
    acc = (correct / total * 100) if total > 0 else 0
    print(f"{class_name:<30}{acc:<15.1f}{correct:<10}{total}")


Mounted at /content/drive
Found 13365 images belonging to 15 classes.
Found 300 images belonging to 15 classes.
Found 750 images belonging to 15 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
418/418 ━━━━━━━━━━━━━━━━━━━━ 3938s 9s/step - accuracy: 0.3667 - loss: 2.0846 - val_accuracy: 0.5133 - val_loss: 1.8108
Epoch 2/5
418/418 ━━━━━━━━━━━━━━━━━━━━ 845s 2s/step - accuracy: 0.6730 - loss: 1.0507 - val_accuracy: 0.5833 - val_loss: 1.5935
Epoch 3/5
418/418 ━━━━━━━━━━━━━━━━━━━━ 891s 2s/step - accuracy: 0.7298 - loss: 0.8677 - val_accuracy: 0.6200 - val_loss: 1.4591
Epoch 4/5
418/418 ━━━━━━━━━━━━━━━━━━━━ 852s 2s/step - accuracy: 0.7570 - loss: 0.7675 - val_accuracy: 0.6833 - val_loss: 1.3709
Epoch 5/5
418/418 ━━━━━━━━━━━━━━━━━━━━ 857s 2s/step - accuracy: 0.7924 - loss: 0.6754 - val_accuracy: 0.6967 - val_loss: 1.3287
750/750 ━━━━━━━━━━━━━━━━━━━━ 200s 265ms/step

Class                         Accuracy (%)   Correct   Total
Aphid                         76.0           38        50
Black Rust                    56.0           28        50
Blast                         88.0           44        50
Brown Rust                    30.0           15        50
Common Root Rot     

In [ ]:
from sklearn.metrics import classification_report

# Use class names in the same order as test_gen.class_indices
class_labels = list(test_gen.class_indices.keys())

# Print precision, recall, F1-score
print(classification_report(y_true, y_pred, target_names=class_labels, digits=3))

In [4]:
# === Imports ===
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from collections import defaultdict
import numpy as np
import os

# === Mount Google Drive (optional) ===
from google.colab import drive
drive.mount('/content/drive')

# === Set paths ===
base_path = '/content/drive/MyDrive/wheat_data'  # adjust if not using Colab
train_path = os.path.join(base_path, 'train')
val_path = os.path.join(base_path, 'valid')
test_path = os.path.join(base_path, 'test')

# === Image settings ===
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5  # Start small; increase later if performance improves

# === Data augmentation for training ===
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# === Generators ===
train_gen = train_datagen.flow_from_directory(
    train_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=True
)

val_gen = val_test_datagen.flow_from_directory(
    val_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

test_gen = val_test_datagen.flow_from_directory(
    test_path, target_size=IMG_SIZE, batch_size=1,
    class_mode='categorical', shuffle=False
)

# === Build MobileNetV2 model ===
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# === Freeze base layers for transfer learning ===
for layer in base_model.layers:
    layer.trainable = False

# === Compile model ===
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# === Train model ===
model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, verbose=1)

# === Evaluate on test set ===
test_gen.reset()
y_pred_probs = model.predict(test_gen, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_gen.classes

# === Get readable class names ===
def clean_class_name(name):
    return name.replace('_test', '').replace('_valid', '').replace('_train', '').replace('_', ' ').title()

true_class_names = [clean_class_name(name) for name in test_gen.class_indices.keys()]

# === Compute per-class accuracy ===
correct_per_class = defaultdict(int)
total_per_class = defaultdict(int)

for i in range(len(y_true)):
    true_class = y_true[i]
    pred_class = y_pred[i]
    class_name = true_class_names[true_class]
    total_per_class[class_name] += 1
    if pred_class == true_class:
        correct_per_class[class_name] += 1

# === Print accuracy per class ===
print(f"\n{'Class':<30}{'Accuracy (%)':<15}{'Correct':<10}{'Total'}")
for class_name in sorted(true_class_names):
    correct = correct_per_class[class_name]
    total = total_per_class[class_name]
    acc = (correct / total * 100) if total > 0 else 0
    print(f"{class_name:<30}{acc:<15.1f}{correct:<10}{total}")

Mounted at /content/drive
Found 13365 images belonging to 15 classes.
Found 300 images belonging to 15 classes.
Found 750 images belonging to 15 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
418/418 ━━━━━━━━━━━━━━━━━━━━ 4833s 11s/step - accuracy: 0.3091 - loss: 2.2816 - val_accuracy: 0.4367 - val_loss: 1.6882
Epoch 2/5
418/418 ━━━━━━━━━━━━━━━━━━━━ 1016s 2s/step - accuracy: 0.6073 - loss: 1.2731 - val_accuracy: 0.5100 - val_loss: 1.5119
Epoch 3/5
418/418 ━━━━━━━━━━━━━━━━━━━━ 1034s 2s/step - accuracy: 0.6515 - loss: 1.0942 - val_accuracy: 0.6033 - val_loss: 1.3716
Epoch 4/5
418/418 ━━━━━━━━━━━━━━━━━━━━ 1025s 2s/step - accuracy: 0.6823 - loss: 1.0002 - val_accuracy: 0.5967 - val_loss: 1.3685
Epoch 5/5
418/418 ━━━━━━━━━━━━━━━━━━━━ 1013s 2s/step - accuracy: 0.6965 - loss: 0.9464 - val_accuracy: 0.5967 - val_loss: 1.3420
750/750 ━━━━━━━━━━━━━━━━━━━━ 233s 308ms/step

Class                         Accuracy (%)   Correct   Total
Aphid                         66.0           33        50
Black Rust                    46.0           23        50
Blast                         96.0           48        50
Brown Rust                    24.0           12        50
Common Root Rot

In [5]:
from sklearn.metrics import classification_report

# Use class names in the same order as test_gen.class_indices
class_labels = list(test_gen.class_indices.keys())

# Print precision, recall, F1-score
print(classification_report(y_true, y_pred, target_names=class_labels, digits=3))


                           precision    recall  f1-score   support

               aphid_test      0.569     0.660     0.611        50
          black_rust_test      0.548     0.460     0.500        50
               blast_test      0.706     0.960     0.814        50
          brown_rust_test      0.279     0.240     0.258        50
     common_root_rot_test      0.722     0.780     0.750        50
fusarium_head_blight_test      0.808     0.420     0.553        50
             healthy_test      0.055     0.060     0.057        50
         leaf_blight_test      0.667     0.480     0.558        50
              mildew_test      0.692     0.540     0.607        50
                mite_test      0.556     0.300     0.390        50
            septoria_test      0.750     0.840     0.792        50
                smut_test      0.619     0.780     0.690        50
            stem_fly_test      0.974     0.740     0.841        50
            tan_spot_test      0.465     0.400     0.430     